# Rain Alert

Project idea comes from [Automate the Boring Stuff with Python](https://automatetheboringstuff.com).

### Launch the Browser with webbrowser Module
The webbrowser module's ```open()``` function can launch a new browser to a specified URL. This is about the only thing the webbrowser module can do, and it helps the launch the weather.com page of my zipcode.

In [ ]:
import webbrowser
webbrowser.open('https://weather.com/weather/hourbyhour/l/10171:4:US')

### Download a Web Page with the ```requests.get()``` Function

The ```requests.get()``` function takes a string of a URL to download, which returns a Response object.

The ```raise_for_status()``` method on this Response object provides a simple way to check download status. An exception will be raised if there was an error doanloading the file and will do nothing otherwise. The ```raise_for_status()``` method is a good way to ensure that the program halts in the event of a bad download.

In [2]:
import requests

res =  requests.get('https://weather.com/weather/hourbyhour/l/10171:4:US')
print(type(res))
res.raise_for_status()

<class 'requests.models.Response'>


### Parsing HTML with the BeautifulSoup Module

Beautiful Soup is a modeule for extracting information from an HTML page. The ```bs4.BeautifulSoup()``` function returns is a BeautifulSoup object.

In [8]:
import bs4

weather = bs4.BeautifulSoup(res.text, 'lxml')
type(weather)

bs4.BeautifulSoup

You can view the source HTML with Browser's Developer Tools, and extract the weather information of your need. 
I need weather of the next 10 hours so I did the following. Since the order of the span elements vary by days, I just grabbed all weather elements of next 10 hours.

In [7]:
spanElems = weather.select('span')

next10 = []
for i in range(0,120):
    next10.append(spanElems[101 + i].get_text())

### Send SMS with Twilio's REST API

I used Twilio's REST API to send outgoing SMS messages from my Twilio phone number. For my free trial account, I can send messages to the number I signed up Twilio with. You can always upgrade your account to send SMS to any phone number. Here are some helpful instructions from [Twilio Help](https://www.twilio.com/docs/sms/send-messages#send-an-sms-with-twilios-api).

In [ ]:
def twilioAlert():
    
    from twilio.rest import Client
    client = Client("your_twilio_account_SID", "your_auth_token")

    # the "from_" number is your Twilio number 
    # the "to" number is the one you signed up for Twilio with, 
    client.messages.create(to = "+1XXXXXXXXXX", 
                           from_= "+1XXXXXXXXXX", 
                           body = "Rain Alert!")

In [ ]:
rainCheck = False
for next in next10:
    if "Rain" in next or "Shower" in next:
        rainCheck = True
        
if (rainCheck):
    twilioAlert()

### Schedule Alert with crontab on Mac OS X 

I prefer to run my script automatically every weekdays at 8:15Am. I use ```crontab```, which is a scheduling tool that will run scripts at regular intervals. 

You can run following coede base on your need from Terminal. Bare in mind that crontab doesn’t read your startup settings from .bashrc for instance, thus you need to let it know where python is installed. 

```15 08 * * * ../../anaconda3/bin/python Python_Projects/rainAlert.py```

###   Le Voilà !

![](https://raw.githubusercontent.com/boneyball/Rain_Alert/master/images/twilio_SMS.jpg?token=ATsBLUPxm_-eCvjC0tgBPbmefCZRVn60ks5ccsMxwA%3D%3D)